In [1]:
using Combinatorics
using ArbNumerics
using GenericLinearAlgebra
using LinearAlgebra
using TimerOutputs
using Optim
using LineSearches
using Tullio
using Nemo
using Random
using Distributions


Welcome to Nemo version 0.28.0

Nemo comes with absolutely no warranty whatsoever



In [14]:
"""
    pseudo_Qprime(h11,tri,cy)
Generates a Q-matrix with similar properties as database Qs
"""
function pseudo_Qprime(h11,tri,cy=1)
    Qh11 = Matrix{Int}(I,h11,h11)
    Q4 = rand(-5:5,(4,h11))
    Q = vcat(Qh11,Q4)
    return Q
end
"""
    pseudo_:prime(h11,tri,cy)
Generates vector of hierarchical instanton scales
"""
function pseudo_Lprime(h11,tri,cy=1)
    Ltemp::Vector{ArbFloat} = sort(vcat([1,0.9,0.8,[rand(ArbFloat)*10. ^-(5*(j-1)) for j=4:h11+4]...]),rev=true)
    return shuffle(Ltemp)
end

pseudo_Lprime

In [3]:
"""
    vacua_out(h11,Q,L)
Computes analytic estimate of number of vacua as a ratio of the volumes of the 
exact symmetries of the potential (θparallel) and the approximate symmetries of the leading
potential inv(Qtilde)

Returns:
 - number of vacua
 - volume of potential
 - volume of lattice containing minima in fundamental domain
"""
function vacua_out(h11::Int,QV::Matrix,LV::Vector)
    S, T, U = snf_with_transform(matrix(Nemo.ZZ,QV))
    Tparallel = Int.(inv(T)[:,1:h11])
    θparalleltest = inv(transpose(QV) * QV) * transpose(QV) * Tparallel
    P = h11+4
    while rank(QV[1:P,:])!=h11
        P+=1
    end
    Lp, Qp = LV[1:P], QV[1:P,:]
    Qtilde = Qp[1,:]
    Qdtilde = zeros(size(Qp[1,:]))
    for i=2:size(Qp,1)
        S = MatrixSpace(Nemo.ZZ, size(Qtilde,1), (size(Qtilde,2)+1))
        m = S(hcat(Qtilde,Qp[i,:]))
        (d,bmat) = Nemo.nullspace(m)
        if d == 0
            Qtilde = hcat(Qtilde,Qp[i,:])
        else
            Qdtilde = hcat(Qdtilde,Qp[i,:])
        end
    end
    vacua = round(abs(det(θparalleltest)) / det(inv(Qtilde)))
    return abs(vacua), Rational.(round.(θparalleltest; digits=10)), Qtilde
end

vacua_out

In [4]:
###################################
#### Code for profiling ###########
###################################

"""
    minimiser_timing(h11,tri,cy,LV,QV,x0,algo)
Optimization function for a particular geometry labelled by `h11`, `tri`, and `cy`.  Takes instanton scales, `Λ`, 
and instanton charges, `Q`, as inputs, as well as some initialisation coordinates `x0`.  Requires specification
of optimisation algorithm _with hessian_, `algo`, _i.e._ `Newton()` or `BFGS()`

Also computes analytic estimate of number of vacua.

Returns:
 - Sign of location of minimum found (mod 2π)
 - log10 of location of minimum found (mod 2π)
 - Matrix of approximate locations of minima (analytic)
"""
function minimiser_timing(h11::Int,tri::Int,cy::Int,LV::Vector,QV::Matrix{Int},x0::Vector,algo)
    reset_timer!()
    """
        vacua_out(h11,Q,L)
    Computes analytic estimate of number of vacua as a ratio of the volumes of the 
    exact symmetries of the potential (θparallel) and the approximate symmetries of the leading
    potential inv(Qtilde)
    
    Returns:
     - number of vacua
     - volume of potential
     - volume of lattice containing minima in fundamental domain
    """
    @timeit "vacua_analytic" function vacua_out(h11::Int,QV::Matrix,LV::Vector)
        S, T, U = snf_with_transform(matrix(Nemo.ZZ,QV))
        Tparallel = Int.(inv(T)[:,1:h11])
#         Tparallel = convert(Matrix{Int},Tparallel)
        θparalleltest = inv(transpose(QV) * QV) * transpose(QV) * Tparallel
        P = h11+4
        while rank(QV[1:P,:])!=h11
            P+=1
        end
        Lp, Qp = LV[1:P], QV[1:P,:]
        Qtilde = Qp[1,:]
        Qdtilde = zeros(size(Qp[1,:]))
        for i=2:size(Qp,1)
            S = MatrixSpace(Nemo.ZZ, size(Qtilde,1), (size(Qtilde,2)+1))
            m = S(hcat(Qtilde,Qp[i,:]))
            (d,bmat) = Nemo.nullspace(m)
            if d == 0
                Qtilde = hcat(Qtilde,Qp[i,:])
            else
                Qdtilde = hcat(Qdtilde,Qp[i,:])
            end
        end
        vacua = round(abs(det(θparalleltest)) / det(inv(Qtilde)))
        return abs(vacua), Rational.(round.(θparalleltest; digits=10)), Qtilde
    end
    Qx = zeros(ArbFloat,size(QV,1));
    """
        fitness(x)
    This is the potential that is optimized at with some initial starting point x
    """
    @timeit "V" function fitness(x::Vector)
        if size(x,1)!=h11
            return "Error, check x"
        end
        V = dot(LV,(1 .- cos.(LinearAlgebra.mul!(Qx,QV,x))))
        return V
    end
    """
        g!(x)
    Updatable gradient of the potential at a point x for optimiser
    """
    @timeit "gradient" function g!(storage::Vector, x::Vector)
        storage .= vcat([dot(LV,QV[:,i] .* sin.(LinearAlgebra.mul!(Qx,QV,x))) for i ∈ 1:h11]...)
    end
    """
        hess(x)
    Hessian evaluated at x to verify eigvals(H(x))>0
    """
    @timeit "hess_check" function hess(x::Vector)
        grad2 = zeros(ArbFloat,(h11,h11))
        hind1 = [[x,y]::Vector{Int64} for x=1:h11,y=1:h11 if x>=y]
        grad2_temp = zeros(ArbFloat,size(hind1,1))
        grad2_temp1 = zeros(Float64,size(LV,1),size(hind1,1))
        @tullio Qx[c] = QV[c,i] * x[i]
        @tullio grad2_temp1[c,k] = @inbounds(begin
        i,j = hind1[k]
                QV[c,i] * QV[c,j] * cos.(Qx[c]) end) grad=false fastmath=false
        @tullio grad2_temp[k] = grad2_temp1[c,k] * LV[c]
        @inbounds for i=1:size(hind1,1)
            j,k = hind1[i]
            grad2[j,k] = grad2_temp[i]
        end
        hessfull = Hermitian(grad2 + transpose(grad2) - Diagonal(grad2))
    end
    """
        hess!(x)
    Updatable version of hessian for optimiser
    """
    @timeit "hess_test" function hess!(hessian::Matrix, x::Vector)
        grad2 = zeros(ArbFloat,(h11,h11))
        hind1 = [[x,y]::Vector{Int64} for x=1:h11,y=1:h11 if x>=y]
        grad2_temp = zeros(ArbFloat,size(hind1,1))
        grad2_temp1 = zeros(Float64,size(LV,1),size(hind1,1))
        @timeit "Qx" begin 
            @tullio Qx[c] = QV[c,i] * x[i]
        end
        @timeit "grad2Q" begin
            @tullio grad2_temp1[c,k] = @inbounds(begin
        i,j = hind1[k]
                QV[c,i] * QV[c,j] * cos.(Qx[c]) end) grad=false
        end
        @timeit "grad2LQ" begin @tullio grad2_temp[k] = grad2_temp1[c,k] * LV[c]
        end
        @timeit "grad2" @inbounds for i=1:size(hind1,1)
            j,k = hind1[i]
            grad2[j,k] = grad2_temp[i]
        end
        @timeit "storage" hessian .= grad2 + transpose(grad2) - Diagonal(grad2)
    end
    """
        grad(x)
    Gradient of potential at x for verifying G(x)>0 at minimum
    """
    grad(x) = vcat([dot(LV,QV[:,i] .* sin.(LinearAlgebra.mul!(Qx,QV,x))) for i ∈ 1:h11]...)
    #################
    ### Optimiser ###
    #################
    @timeit "optim" res = optimize(fitness,g!,hess!,
                x0, algo,
                Optim.Options(x_tol =minimum(abs.(LV)),g_tol =minimum(abs.(LV)),successive_f_tol = 5))
    Vmin = Optim.minimum(res)
    xmin = Optim.minimizer(res)
    #####################
    ## Verify minimum ###
    ### and output ######
    #####################
    if minimum(eigen(hess(xmin)).values) > 0 && maximum(abs.(grad(xmin))) < 1e-10
        @timeit "vacua" vacua, θparalleltest, Qtilde = vacua_out(h11, QV, LV)
        @timeit "a" a = mod.(((θparalleltest * xmin)/2π),1)
        a_sign::Vector{Int} = sign.(a)
        a_log::Vector{Float64} = log10.(abs.(a))
        Vmin_sign::Int = sign(Vmin)
        Vmin_log::Float64 = log10(abs(Vmin))
        xmin_log::Vector{Float64} = log10.(abs.(xmin))
        xmin_sign::Vector{Int} = sign.(xmin)
        print_timer()
        a_sign, a_log, θparalleltest
    end
end



minimiser_timing

In [5]:
"""
    minimiser(h11,tri,cy,LV,QV,x0,algo)
Optimization function for a particular geometry labelled by `h11`, `tri`, and `cy`.  Takes instanton scales, `Λ`, 
and instanton charges, `Q`, as inputs, as well as some initialisation coordinates `x0`.  Requires specification
of optimisation algorithm _with hessian_, `algo`, _i.e._ `Newton()` or `BFGS()`

Also computes analytic estimate of number of vacua.

Returns:
 - Sign of location of minimum found (mod 2π)
 - log10 of location of minimum found (mod 2π)
 - Matrix of approximate locations of minima (analytic)
"""
function minimiser(h11::Int,tri::Int,cy::Int,LV::Vector,QV::Matrix{Int},x0::Vector,algo)
    
    Qx = zeros(ArbFloat,size(QV,1));
    """
        fitness(x)
    This is the potential that is optimized at with some initial starting point x
    """
    function fitness(x::Vector)
        if size(x,1)!=h11
            return "Error, check x"
        end
        V = dot(LV,(1 .- cos.(LinearAlgebra.mul!(Qx,QV,x))))
        return V
    end
    """
        g!(x)
    Updatable gradient of the potential at a point x for optimiser
    """
    function g!(storage::Vector, x::Vector)
        storage .= vcat([dot(LV,QV[:,i] .* sin.(LinearAlgebra.mul!(Qx,QV,x))) for i ∈ 1:h11]...)
    end
    """
        hess(x)
    Hessian evaluated at x to verify eigvals(H(x))>0
    """
    function hess(x::Vector)
        grad2 = zeros(ArbFloat,(h11,h11))
        hind1 = [[x,y]::Vector{Int64} for x=1:h11,y=1:h11 if x>=y]
        grad2_temp = zeros(ArbFloat,size(hind1,1))
        grad2_temp1 = zeros(Float64,size(LV,1),size(hind1,1))
        @tullio Qx[c] = QV[c,i] * x[i]
        @tullio grad2_temp1[c,k] = @inbounds(begin
        i,j = hind1[k]
                QV[c,i] * QV[c,j] * cos.(Qx[c]) end) grad=false fastmath=false
        @tullio grad2_temp[k] = grad2_temp1[c,k] * LV[c]
        @inbounds for i=1:size(hind1,1)
            j,k = hind1[i]
            grad2[j,k] = grad2_temp[i]
        end
        hessfull = Hermitian(grad2 + transpose(grad2) - Diagonal(grad2))
    end
    """
        hess!(x)
    Updatable version of hessian for optimiser
    """
    function hess!(hessian::Matrix, x::Vector)
        grad2 = zeros(ArbFloat,(h11,h11))
        hind1 = [[x,y]::Vector{Int64} for x=1:h11,y=1:h11 if x>=y]
        grad2_temp = zeros(ArbFloat,size(hind1,1))
        grad2_temp1 = zeros(Float64,size(LV,1),size(hind1,1))
        @tullio Qx[c] = QV[c,i] * x[i]
        @tullio grad2_temp1[c,k] = @inbounds(begin
        i,j = hind1[k]
                QV[c,i] * QV[c,j] * cos.(Qx[c]) end) grad=false
        @tullio grad2_temp[k] = grad2_temp1[c,k] * LV[c]
        @inbounds for i=1:size(hind1,1)
            j,k = hind1[i]
            grad2[j,k] = grad2_temp[i]
        hessian .= grad2 + transpose(grad2) - Diagonal(grad2)
        end
    end
    """
        grad(x)
    Gradient of potential at x for verifying G(x)>0 at minimum
    """
    grad(x) = vcat([dot(LV,QV[:,i] .* sin.(LinearAlgebra.mul!(Qx,QV,x))) for i ∈ 1:h11]...)
    #################
    ### Optimiser ###
    #################
    res = optimize(fitness,g!,hess!,
                x0, algo,
                Optim.Options(x_tol =minimum(abs.(LV)),g_tol =minimum(abs.(LV)),successive_f_tol = 5))
    Vmin = Optim.minimum(res)
    xmin = Optim.minimizer(res)
    #####################
    ## Verify minimum ###
    ### and output ######
    #####################
    if minimum(eigen(hess(xmin)).values) > 0 && maximum(abs.(grad(xmin))) < 1e-10
        vacua, θparalleltest, Qtilde = vacua_out(h11, QV, LV)
        a = (θparalleltest * xmin)/2π
        
        a_sign = Int.(sign.(a))
        a_log = [al ==0 ? 0 : Float64.(log10.(abs.(al))) for al in a]
        Vmin_sign = Int(sign(Vmin))
        Vmin_log = Float64(log10(abs(Vmin)))
        xmin_log = Float64.(log10.(abs.(xmin)))
        xmin_sign = Int.(sign.(xmin))
        a_sign, a_log, θparalleltest
    else
        (zeros(h11),zeros(h11),zeros(h11,h11))
    end
end



minimiser

In [20]:
"""
    main_timing(x0)
Function for looping over multiple initialisations (profiler)
"""
function main_timing(x0)
    h11,tri,cy = 10,1,1
    h11list = hcat([vcat(h11,tri,cy) for _=1:n]...)
    algo_hz = Newton(alphaguess = LineSearches.InitialHagerZhang(α0=1.0), linesearch = LineSearches.HagerZhang());
    return minimiser_timing(h11,tri,cy,LV,QV,x0,algo_hz)
end
"""
    main_test(x0)
Function for looping over multiple initialisations
"""
function main_test(x0)
    h11list = hcat([vcat(h11,tri,cy) for _=1:n]...)
    algo_hz = Newton(alphaguess = LineSearches.InitialHagerZhang(α0=1.0), linesearch = LineSearches.HagerZhang());
    return minimiser(h11,tri,cy,LV,QV,x0,algo_hz)
end

main_test

In [21]:
h11,tri,cy = 4,1,1
n=100 ##number of initialisations
x0 = hcat([rand(Uniform(0,2π),h11) .* rand(ArbFloat,h11) for _=1:n]...);
LV = pseudo_Lprime(h11,tri,cy)
QV = pseudo_Qprime(h11,tri,cy)

8×4 Matrix{Int64}:
  1   0   0   0
  0   1   0   0
  0   0   1   0
  0   0   0   1
 -3   5  -3  -1
 -2  -2  -2   4
 -3  -2  -3   0
  0   1  -4  -3

In [22]:
@time res = mapslices(main_test,x0,dims=1);

  3.631709 seconds (33.09 M allocations: 877.348 MiB, 23.82% gc time, 29.06% compilation time)


In [25]:
### This tells us, within some tolerance, how many unique minima 
### have been found.  Some artefacts remain so check by eye in final set of minima
unique_mins = unique(mod.(round.(hcat([10 .^res[i][2] for i=1:size(res,2)]...),digits=10),1),dims=2)

4×52 Matrix{Float64}:
 0.0        0.0        0.0        0.0  …  0.0        0.0       0.0
 0.357145   0.857145   0.0714265  0.5     0.357145   0.142855  0.214285
 7.2866e-6  7.2866e-6  5.8425e-6  0.0     7.2866e-6  0.999993  0.999997
 0.785705   0.285705   0.35715    0.5     0.214295   0.285705  0.928567

In [24]:
## Check analytic computation of number of vacua
## is equal to number of unique minima found by optimization
vacua = vacua_out(h11,QV,LV)[1]
if vacua != size(unique_mins,2)
    vacua
else
    true
end

1.0